<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LSTM使用模式" data-toc-modified-id="LSTM使用模式-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LSTM使用模式</a></span></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>参考资料</a></span></li></ul></div>

# LSTM使用模式

1. 定义LSTM结构
2. 初始化LSTM的状态
3. 定义num_step, 避免梯度消散
4. 进行前向计算
5. 计算最终输出
6. 计算损失函数
7. 训练模型

其中, 还有几个地方不是很明白(如何打印具有具体的数据)

In [1]:
import tensorflow as tf
import numpy as np

In [9]:
tf.reset_default_graph()

HIDDEN_SIZE = 2     # 定义隐含层的规模
batch_size = 2      # 定义batch的大小
num_steps = 5       # 定义数据截断的长度

# 定义数据
# X shape: [batch_num, batch_size, embedding_size], embedding_size表示输入数据的维度
# 其中batch_num:表示batch的个数, batch_size:表示batch的大小, embedding_size表示输入的维度
X = np.random.random(size=[5, 2, 1])
Y = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

# 定义一个基本的LSTM结构
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)

# 将LSTM中的状态初始化为全0的数组
# state shape: [batch_size, HIDDEN_SIZE]
state = lstm_cell.zero_state(batch_size, tf.float64)

# 定义用于输出的全连接层[2, 1]
w_output = np.random.random(size=[HIDDEN_SIZE, 1])
b_output = 0.1

# 定义损失函数
loss = 0.0

with tf.Session() as sess:
    # 初始化变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    #tf.global_variables_initializer().run()
    
    # 按时间顺序循环执行神经网络的前向传播过程:运行一个截断长度
    for i in range(num_steps):
        # 在第一个时刻声明LSTM结构中使用的变量，在这之后的时刻都需要复用之前定义好的变量
        if i > 0:
            tf.get_variable_scope().reuse_variables()

        # 每一步处理时间序列中的一个时刻。
        # 将当前输入(current_input)和前一个时刻状态(state)传入定义的LSTM结构
        # 得到当前LSTM结构的输出lstm_output和更新后的状态state
        # 输入的维度为[batch_size, embedding_size], state的维度为[batch_size, HIDDEN_SIZE]
        # lstm_output的维度为[batch_size, HIDDEN_SIZE]; state包含(c, h)，它们的维度为[batch_size, HIDDEN_SIZE]
        lstm_output, state = lstm_cell(tf.convert_to_tensor(X[i]), state)

        # 将当前时刻LSTM结构的输出传入一个全连接层得到最后的输出(此处没有设置激活函数)
        # lstm shape: [batch_size, HIDDEN_SIZE], w_output shape:[HIDDEN_SIZE, output_size]
        final_output = np.dot(lstm_output, w_output) + b_output

        # 计算当前时刻的损失
        #loss += calc_loss(final_output, expected_output)

        # 输出每个时刻的信息(如何打印数据)
        print("X: \n", X[i])
        print("lstm_output: \n", lstm_output)
        print("state:\n", state)
        print("final_output:\n", final_output)
        print("\n")

X: 
 [[0.8197666 ]
 [0.17715495]]
lstm_output: 
 Tensor("basic_lstm_cell/Mul_2:0", shape=(2, 2), dtype=float64)
state:
 LSTMStateTuple(c=<tf.Tensor 'basic_lstm_cell/Add_1:0' shape=(2, 2) dtype=float64>, h=<tf.Tensor 'basic_lstm_cell/Mul_2:0' shape=(2, 2) dtype=float64>)
final_output:
 [[<tf.Tensor 'add:0' shape=(2, 2) dtype=float64>]
 [<tf.Tensor 'add_1:0' shape=(2, 2) dtype=float64>]]


X: 
 [[0.10726922]
 [0.34934291]]
lstm_output: 
 Tensor("basic_lstm_cell/Mul_5:0", shape=(2, 2), dtype=float64)
state:
 LSTMStateTuple(c=<tf.Tensor 'basic_lstm_cell/Add_3:0' shape=(2, 2) dtype=float64>, h=<tf.Tensor 'basic_lstm_cell/Mul_5:0' shape=(2, 2) dtype=float64>)
final_output:
 [[<tf.Tensor 'add_2:0' shape=(2, 2) dtype=float64>]
 [<tf.Tensor 'add_3:0' shape=(2, 2) dtype=float64>]]


X: 
 [[0.96430421]
 [0.61233351]]
lstm_output: 
 Tensor("basic_lstm_cell/Mul_8:0", shape=(2, 2), dtype=float64)
state:
 LSTMStateTuple(c=<tf.Tensor 'basic_lstm_cell/Add_5:0' shape=(2, 2) dtype=float64>, h=<tf.Tensor 

# 参考资料

1. 《TensorFlow实战google深度学习框架》